## **MinHashLSH**

### In local system memory implementation:

In [1]:
import pandas as pd
import numpy as np

In [2]:
class InMemoryMinHashLSH:
    def __init__(self, documents, k=5):
        self.documents = documents
        self.shingles = None
        self.signatures = None
        self.buckets = None
        self.num_bands = 8
        self.k = k
        self.num_perms = 128
    
    def shingling(self, documents=pd.DataFrame([""]), k=5):
        if documents.any().any() == "":
            self.documents = documents

        shingles = set()
        doc_shingles = set()
        for doc in self.documents["text"]:
            for i in range(len(doc) - k + 1):
                shingle = doc[i:i+k]
                shingles.add(shingle)
                doc_shingles.add(shingle)
        shingles = list(shingles)
        
        boolean_vectors = np.full((len(self.documents), len(shingles)), False, dtype=bool)
        for i, doc in enumerate(self.documents["text"]):
            for j, shingle in enumerate(shingles):
                if shingle in doc:
                    boolean_vectors[i, j] = True
        return pd.DataFrame(boolean_vectors, columns=shingles).transpose()
        
    def minhashing(self, shingles_bvs, num_perm=128):
        signatures = []
        for _ in range(0, num_perm):
            hash_funcs = np.random.permutation(shingles_bvs.shape[0])
            signature_row = []
            for j in range(0, shingles_bvs.shape[1]):
                for hash in hash_funcs:
                    if shingles_bvs.iloc[hash, j]:
                        signature_row.append(hash)
                        break
            signatures.append(signature_row)
        return pd.DataFrame(signatures)
    
    def locality_sensitive_hashing(self, signatures, num_bands=8):
        self.num_bands = num_bands
        buckets = {}
        for doc_id in signatures:
            sig = signatures[doc_id]
            for i in range(0, len(sig), self.num_bands):
                band = hash(tuple(sig[i:i+self.num_bands]))
                if band in buckets:
                    buckets[band].add(doc_id)
                else:
                    buckets[band] = {doc_id}
        return buckets

    def run(self, **kwargs):
        if "documents" in kwargs:
            self.documents = kwargs["documents"]
        if "bands" in kwargs:
            self.num_bands = kwargs["bands"]
        if "num_perms" in kwargs:
            self.num_perms = kwargs["num_perms"]
        if "k" in kwargs:
            self.k = kwargs["k"]
        bitvecs = self.shingling(self.documents, self.k)

        # The regular permutation variant of the minhashing algorithm is used
        # assuming that the amount of data process is relatively small to fit in memory
        # of course we'll utilize the row hashing variant in the spark implementation
        self.signatures = self.minhashing(bitvecs, self.num_perms) 
        self.buckets = self.locality_sensitive_hashing(self.signatures, self.num_bands)
        return self.buckets
    
    def __jaccard_similarity(self, a, b):
        return len(a & b) / len(a | b)
    
    def approximateNearestNeighbors(self, key, n):
        n = 1 if n > 1 else n
        sig = self.signatures[key]
        similar_docs = {}
        for i in range(0, len(sig), self.num_bands):
            band_hash = hash(tuple(sig[i:i+self.num_bands]))
            if band_hash in self.buckets:
                for doc_id in self.buckets[band_hash]:
                    if doc_id != key:
                        if doc_id in similar_docs:
                            similar_docs[doc_id] += 1
                        else:
                            similar_docs[doc_id] = 1
        similar_docs = {k: v for k, v in sorted(similar_docs.items(), key=lambda item: item[1], reverse=True)}

        most_similar_docs = []
        for doc_id in similar_docs:
            jac_sim = self.__jaccard_similarity(set(self.signatures[key]), set(self.signatures[doc_id]))
            if jac_sim > n:
                most_similar_docs.append((doc_id, jac_sim))
        return most_similar_docs

In [3]:
test_docs = ["This is a test document", "This document is another test document", "This is a test document","This is a test","This is a document", "Hello word"]
docs_df = pd.DataFrame(test_docs, columns=["text"])
in_memory_lsh = InMemoryMinHashLSH(docs_df)
pd.set_option('display.max_columns', None)

# bool_vecs = in_memory_lsh.shingling()
# sigs = in_memory_lsh.minhashing(bool_vecs, 128)
# buckets = in_memory_lsh.locality_sensitive_hashing(sigs)
buckets = in_memory_lsh.run(k=3, bands=4, num_perms=256, documents=docs_df)
# print(buckets)
in_memory_lsh.approximateNearestNeighbors(0, 0.5)

[(2, 1.0), (4, 0.6666666666666666), (3, 0.55), (1, 0.5483870967741935)]

### Spark implementation:

In [4]:
from pyspark.sql import functions as F 
from pyspark.sql import window as W
from pyspark.sql.types import ArrayType, IntegerType, StringType

Auxiliary functions:

In [5]:
###############
# Function name: hash_gen
# Input: (Int) the number of "hash functions" to generate (Also happen to be signature length)
# Output: (List) of tuples of 3 containing the hash functions parameters, example [(1, 3, 5), (2, 4, 6), ...]
# Purpose: Generate the hash functions parameters for the minhashing process. This function is part of the minhashing method
###############
def hash_gen(num_hashes=128):
   hashes = []
   for i in range(num_hashes):
       hash = [np.random.randint(1, 1000), np.random.randint(1, 1000), np.random.randint(1, 1000)]
       hashes.append(hash)
   return hashes

###############
# Function name: hash_bands
# Input: (List) of integers, (Int) the number of bands to hash the integers into (Defaulted to 2 if longer than signature length)
# Output: (List) of integers of the hashed bands
# Purpose: Hash the signature into bands for the locality sensitive hashing process. This function is part of the locality sensitive hashing method RDD
###############
def hash_bands(row, num_bands=8):
    arr_size = int(len(row) / num_bands)
    arr_size = 2 if arr_size == 0 else arr_size
    bands = []
    for i in range(0, len(row), arr_size):
        bands.append(hash(tuple(row[i:i+arr_size])))
    return bands

###############
# Function name: minhash_udf
# Input: (List) of squashed tuples containing the row id and the boolean vector, (Int) the signature length
# Output: (List) of lists containing the minhashed values
# Purpose: Minhash the boolean vectors into signatures. This function is part of the minhashing method RDD
###############
def minhash_udf(row, hash_funcs):#, sig_length=128):
    # Generating the "infinite" matrix (can't use np.inf as it's not supported by spark, class error w/e)
    final_ans = [[-2 for _ in range(len(row[0][1]))] for _ in range(len(hash_funcs))]
    for row in row:
        row_id = row[0]
        row_vals = row[1]
        for i in range(len(hash_funcs)):
            # Hashing the row id
            curr_hash = ((hash_funcs[i][0] * row_id) + hash_funcs[i][1]) % hash_funcs[i][2]
            for j in range(len(row_vals)):
                if row_vals[j]:
                    # Minhashing with the "infinite" matrix
                    final_ans[i][j] = curr_hash if final_ans[i][j] == -2 else min(final_ans[i][j], curr_hash)
    return final_ans

In [174]:
class SparkMinHashLSH:
    def __init__(self, documents, k=5):
        self.documents = documents
        self.signatures = None
        self.num_bands = 8
        self.k = k
        self.num_perms = 128
    
    def shingling(self, **kwargs):
        self.documents = kwargs["documents"] if "documents" in kwargs else self.documents
        
        # High k values would result in document skipping if the document is smaller than the shingle size
        shingle_size = 5 if "k" not in kwargs else kwargs["k"]

        # The monotonically increasing ID function only generates random unique IDs
        # Thus requiring us to use the row_number function instead
        # The following code is essentially creating a "window" of ordered from 1, the "lit"(literal) function acts the same way as passing an arg to a function 
        id_doc_df = self.documents.withColumn("docID", F.row_number().over(W.Window.orderBy(F.lit(1))))
        shingles = (id_doc_df.rdd.map(lambda x: (x[0],x[1])) 
                   .map(lambda x: list(set([(x[1], x[0][i:i+shingle_size]) for i in range(len(x[0]) - shingle_size + 1)]))) # Shingling with k size shingle
                   .flatMap(lambda x: x) # DF formatting, from shape [(docID, shingle), ...] to (docID, shingle), aka unknown data shape to 2 columns
                   .toDF(["docID", "shingles"])
                    )
                    
        return (shingles.groupBy("shingles")
                .pivot("docID") # Rotating the DF based on docIDs
                .agg(F.lit(True)) # Create a true column and aggregate on each present intersected shingles corresponding to the docID
                .fillna(False)) # Fill the NaNs left by the aggregation process with False
    
    def minhashing(self, **kwargs):
        if "bool_vecs" not in kwargs:
            raise ValueError("Boolean Vectors not provided")
        bool_vecs = kwargs["bool_vecs"]
        sig_length = 128 if "sig_length" not in kwargs else kwargs["sig_length"]
        
        # Col numbering nothing exciting
        bool_vecs = bool_vecs.withColumn("id", F.row_number().over(W.Window.orderBy(F.lit(1)))) 

        # Grouping the bool vecs into lists for processing
        zipped_bv = bool_vecs.select("id", F.array([F.col(x) for x in bool_vecs.columns if x != "id" and x != "shingles"]).alias("vals")) 

        # Defining the return type so it doesn't become string cuz spark doesnt know how to process lol   
        typed_minhash_udf = F.udf(minhash_udf, ArrayType(ArrayType(IntegerType()))) 
        
        # Generating the hash functions and transforming it into a spark udf friendly format for parameter passing
        # This has to be done separately as spark reconstructs the table from scratch everytime show() is called 
        # or anything that requires the table to be "materialized", side effect of that is random signatures every time show is called
        hash_funcs = hash_gen(sig_length)
        transformed_hash_funcs = F.array([F.array(F.lit(x[0]), F.lit(x[1]), F.lit(x[2])) for x in hash_funcs])
        
        # Black magic, jk, continue reading at the above auxiliary udf func named "minhash_udf"
        sigs = zipped_bv.agg(typed_minhash_udf(F.collect_list(F.struct(F.col("id"), F.col("vals"))), transformed_hash_funcs).alias("sigs")) 
        
        # Convert it to a more "expected" and "familiar" format of signatures per col, and renaming the columns
        sig = (sigs
                .select(F.explode("sigs"))
                .rdd
                .flatMap(lambda x: x)
                .toDF()
                )
        # Column renaming 
        return sig.select([F.col(x).alias(f"{x.strip('_')}") for x in sig.columns])
    
    def locality_sensitive_hashing(self, **kwargs):
        if "sigs" not in kwargs:
            raise ValueError("Signatures not provided")
        
        num_bands = 10 if "num_bands" not in kwargs else kwargs["num_bands"]
        sigs = kwargs["sigs"]

        # Defining the aggregate expression, basically squashing the columns into lists
        squash_sigs = [F.collect_list(F.col(x)).alias(x) for x in sigs.columns] 
        squashed_sigs = (sigs
                        .agg(*squash_sigs)
                        .select(F.explode(F.array([F.array(F.col(x)) for x in sigs.columns]))
                       .alias("sigs"))
                       .withColumn("doc_id", F.row_number().over(W.Window.orderBy(F.lit(1)))))
        bands_list = squashed_sigs.rdd.map(lambda x: (x[1],hash_bands(x[0][0], num_bands)))
        return  (bands_list
                   .toDF(["doc_id", "bands"])
                   .select("doc_id", F.explode("bands").alias("buckets"))
                   .groupBy("buckets")
                   .agg(F.collect_set("doc_id").alias("doc_ids")))

    def run(self, **kwargs):
        if "documents" in kwargs:
            documents = kwargs["documents"]
        if "bands" in kwargs:
            self.num_bands = kwargs["bands"]
        if "num_perms" in kwargs:
            self.num_perms = kwargs["num_perms"]
        if "k" in kwargs:
            self.k = kwargs["k"]
        bool_vecs = self.shingling(documents=documents, k=self.k)
        signatures = self.minhashing(bool_vecs=bool_vecs, sig_length=self.num_perms)
        self.signatures = signatures
        buckets = self.locality_sensitive_hashing(sigs=signatures, num_bands=self.num_bands)
        return buckets

    def jaccard_similarity(self, first_key, second_key):
        sig_table = self.signatures
        union = sig_table.select(f"{first_key}").union(sig_table.select(f"{second_key}")).distinct().count()
        intersect = sig_table.select(f"{first_key}").intersect(sig_table.select(f"{second_key}")).distinct().count()
        return intersect / union
    
    def approximateNearestNeighbors(self, key, n,):
        
        num_bands = int(self.num_bands)
        selected_sigs = (self.signatures
            .select(f"{key}")
            .agg(F.collect_list(f"{key}"))
            .rdd.map(lambda x: hash_bands(x[0], num_bands))
            .flatMap(lambda x: x)
            .map(lambda x: (x, ))
            .toDF(["buckets"])
        )

        similar_docs = buckets.join(selected_sigs, "buckets")
        doc_id = similar_docs.select(F.explode("doc_ids").alias("doc_id")).distinct().collect()
        similar_docs = []

        # TODO: Implement threading for parallel processing
        for doc in doc_id:
            jac_sim = self.jaccard_similarity(key, doc.doc_id)
            # if jac_sim > n:
            similar_docs.append((doc.doc_id, jac_sim))
        
        return similar_docs 

In [105]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("minHashLSH").getOrCreate()
sc = spark.sparkContext

In [175]:
data = [("This is a test document",), ("This document is another test document",), ("This is a test document",), ("Hello wordello ",), ("Word Hello world",), ("hello", )]
df = spark.createDataFrame(data, ["text"])
spark_lsh = SparkMinHashLSH(df)
# bool_vecs = spark_lsh.shingling(documents=df, k=2)
# sigs = spark_lsh.minhashing(bool_vecs=bool_vecs, sig_length=512)
# buckets = spark_lsh.locality_sensitive_hashing(sigs=sigs, num_bands=8)

# Increase the number of bands and perms to decrease the sensitivity of the LSH, effectively reducing the number of false positives
buckets = spark_lsh.run(documents=df,k=2, bands=8, num_perms=16, document=df)
buckets.show(10)
spark_lsh.approximateNearestNeighbors(1, 0.6)


+--------------------+---------+
|             buckets|  doc_ids|
+--------------------+---------+
| 2000638592450125738|   [5, 4]|
| 1283277135472676147|      [4]|
|-5545844477663614845|      [6]|
|-5657247872280712689|      [2]|
| 4307797553634642394|[5, 6, 4]|
| 4803095124130366729|      [5]|
| 2145580550687776204|   [1, 3]|
|-3381654230262078061|   [5, 2]|
|-6539622585183552344|      [2]|
| 2422434102992159561|[1, 2, 3]|
+--------------------+---------+
only showing top 10 rows



[(1, 1.0), (3, 1.0), (2, 0.5555555555555556)]

In [170]:
# unions = int((spark_lsh.signatures.select("1").union(spark_lsh.signatures.select("2")).distinct().count()))
# intersects = int(spark_lsh.signatures.select("1").intersect(spark_lsh.signatures.select("2")).distinct().count())
# intersects/unions

0.7380952380952381